In [10]:
import matplotlib as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [11]:
def loss_plot(hist):
    train_epochs = list(range(1, len(hist.history['loss']) + 1))
    valid_epochs = list(range(1, len(hist.history['val_loss']) + 1))
    train_losses = hist.history['loss']
    valid_losses = hist.history['val_loss']

    train_min_loss = min(dict(zip(train_epochs, train_losses)).items(), key=lambda x: x[1])
    valid_min_loss = min(dict(zip(valid_epochs, valid_losses)).items(), key=lambda x: x[1])

    plt.plot(train_epochs, hist.history['loss'], linewidth=1, color='blue', label='Training Loss')
    plt.plot(train_min_loss[0], train_min_loss[1], color='black', marker='o',
             label=f'Training Best Epoch - {train_min_loss[0]}')
    plt.plot(valid_epochs, hist.history['val_loss'], linewidth=1, color='red', label='Validation Loss')
    plt.plot(valid_min_loss[0], valid_min_loss[1], color='black', marker='o',
             label=f'Validation Best Epoch - {valid_min_loss[0]}')
    plt.legend(loc='best')
    plt.title(label='Training & Validation Loss')
    plt.xlabel(xlabel='Epochs')
    plt.ylabel(ylabel='Loss')
    plt.show()

In [12]:
def accuracy_plot(hist):
  epochs = list(range(1, len(hist.history['accuracy']) + 1))

  plt.plot(epochs, hist.history['accuracy'], marker='o', linewidth=1, label='Training Accuracy', color='blue')
  plt.plot(epochs, hist.history['val_accuracy'], marker='o', linewidth=1, label='Validation Accuracy', color='red')
  plt.title(label='Training & Validation Accuracy')
  plt.legend(loc='best')
  plt.xlabel(xlabel='Epochs')
  plt.ylabel(ylabel='Accuracy')
  plt.show()

In [13]:
model = keras.models.Sequential(layers=[
    Conv2D(filters=32, kernel_size=(3, 3), input_shape=(150, 150, 3),
           padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(rate=0.2),
    Dense(units=256, activation='relu'),
    Dense(units=4, activation='softmax')
])
     


2023-06-07 09:09:25.405671: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 09:09:25.411824: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.
